In [1]:
import numpy as np

from bokeh.io import curdoc
from bokeh.layouts import column, row
from bokeh.models import ColumnDataSource, Slider, TextInput, Toggle, Button, RadioButtonGroup, Paragraph
from bokeh.plotting import figure

import time

useSerial = False

# Set up the CD48 board
if useSerial:
    s = serial.Serial('/dev/cu.usbmodem14101',baudrate=250000) # May need to change the port number
    s.write("S01100\n".encode()) 

# Initialize the data lists
t = [0]
n = [0]

# Set the source of data for our plot
source = ColumnDataSource(data=dict(t=t, n=n))

# Set up plot
plot = figure(plot_height=400, plot_width=800, title="Coincidence Counts",
              tools="crosshair,pan,reset,save,wheel_zoom")

# Make each point of the plot a circle
plot.circle('t', 'n', source=source, line_width=3, line_alpha=0.6)


# Set up START/STOP button
button_group = RadioButtonGroup(
        labels=["START", "STOP"], active=1)

# Set up counting interval slider
intervalSlider = Slider(start=100, end=1000, value=1000, step=1, title="Counting interval")
dt = intervalSlider.value # initialize the variable to store the counting interval

# Set up the reset button
resetButton = Button(label="Reset")

# Set up time average value text box
avgText = Paragraph(text="100", width=400, height=40)



# Define a fuction to call when the current document (curdoc) has periodic callback on
def update_data():
    
    dt = intervalSlider.value
    
    # Generate the new curve
    # Append a new t value (based on the value of the interval slider) to list t
    t.append(t[-1]+dt)
    
    # Append the most recently received count to list n
    if useSerial:
        # Request data from the CD48 counter
        s.write("c\n".encode())
        serialData = s.readline()
        data = [int(x) for x in serialData.decode('ascii').rstrip().split(' ')]
        n.append(data[0])
    else:
        # generate random numbers from 0 to dt (in miliseconds)
        n.append(np.random.randint(0, dt))
        
    # Set the data source
    source.data = dict(t=t, n=n)
    
    # Set up the graph to start scrolling when x_range is filled
    # The plot will display the most recent 10 points, 
    # but you can manually scroll the graph back to see earlier data points.
    # No longer works after dragging the graph. Need to figure out a solution
    plot.x_range.follow = "end"
    plot.x_range.follow_interval = dt*10
    
    # Draw the new points and connect them with a line
    plot.line('t', 'n', source=source, line_width=3, line_alpha=0.6)
    plot.circle('t', 'n', source=source, line_width=3, line_alpha=0.6)
    
    # Update the average count in the text box
    avgText.text = "Average Count: %d " % (np.mean(n))
    
# Define a variable callback_id, which is attached to the periodic_callback function
callback_id = None


# Define a function to be called when the START/STOP button changes value
def change_periodic_callback(new):
    # use global variable
    global callback_id
    
    # if the START button is on
    if button_group.active == 0:
        
        # Add a periodic_callback, the callback interval is stored in the variable dt
        dt = intervalSlider.value
        callback_id = curdoc().add_periodic_callback(update_data, dt)
    
    # else if the STOP button is on
    elif button_group.active == 1:
        
        # Remove the periodic callback
        curdoc().remove_periodic_callback(callback_id)
        
    
# Define a function to be called when the reset button is clicked
def reset_plot():
    # Change the lists t and n back to [0]
    global t
    global n
    t = [0]
    n = [0]
    
    # Set the data source
    source.data = dict(t=t, n=n)
    
    # Draw the new points and connect them with a line
    plot.line('t', 'n', source=source, line_width=3, line_alpha=0.6)
    plot.circle('t', 'n', source=source, line_width=3, line_alpha=0.6)

# When the STAR/STOP button is clicked, call the change_periodic_callback function
button_group.on_click(change_periodic_callback)


# When the reset button is clicked, call the reset_plot function
resetButton.on_click(reset_plot)

# Set up layouts and add to document
inputs = column(button_group, intervalSlider, resetButton, avgText)


# Add plot and widgets to current document
curdoc().add_root(row(inputs, plot, width=1000))
curdoc().title = "coincidenceInterface"
    

